In [ ]:
import os
import pickle
import requests
import pandas as pd
import re

# ───────────────────── TEMPLATE 參數區 ─────────────────────
API_BASE         = 'https://api.worldquantbrain.com'  # API 根 URL
SESSION_PKL      = 'SESSION_PATH'  # Session pickle 路徑
DATASET_ID       = 'univ1'     # Dataset ID，修改為要抓取的 dataset
DELAY            = 1             # delay 參數
INSTRUMENT_TYPE  = 'EQUITY'       # instrumentType 參數
REGION           = 'USA'          # region 參數
UNIVERSE         = 'TOP3000'      # universe 參數
LIMIT            = 20            # 每頁筆數限制
FIELD_PREFIX     = ''             # 篩選前綴：空字串表示擷取所有欄位
OUTPUT_DIR       = 'OUTPUT_PATH'  # 輸出資料夾

# ───────────────────── 以下程式不需修改 ─────────────────────
# 載入 session
with open(SESSION_PKL, 'rb') as f:
    session = pickle.load(f)

# 取得 dataset 詳細以獲取 name
resp_meta = session.get(
    f"{API_BASE}/data-sets/{DATASET_ID}",
    params={
        'delay': DELAY,
        'instrumentType': INSTRUMENT_TYPE,
        'region': REGION,
        'universe': UNIVERSE
    }
)
resp_meta.raise_for_status()
meta = resp_meta.json()

# 清理 dataset 名稱以作檔名
dataset_name = meta.get('name', DATASET_ID)
clean_name = re.sub(r'[^\w\-\s]', '', dataset_name)
clean_name = re.sub(r'\s+', '_', clean_name).strip('_')

# 輸出路徑
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_XLSX = os.path.join(OUTPUT_DIR, f"{clean_name}_fields.xlsx")

# API 抓欄位參數
PARAMS = {
    'dataset.id':       DATASET_ID,
    'delay':            DELAY,
    'instrumentType':   INSTRUMENT_TYPE,
    'region':           REGION,
    'universe':         UNIVERSE,
    'limit':            LIMIT,
    'offset':           0
}

# 擷取資料
records = []
while True:
    resp = session.get(f"{API_BASE}/data-fields", params=PARAMS)
    resp.raise_for_status()
    payload = resp.json()

    items = payload.get('data') or payload.get('results') or []
    if not items:
        break

    for item in items:
        fid      = item.get('id') or item.get('fieldId') or item.get('name')
        desc     = item.get('description') or ''
        ftype    = item.get('type') or ''
        coverage = item.get('coverage')
        if coverage is not None:
            coverage = f"{coverage*100:.0f}%" if isinstance(coverage, float) else str(coverage)
        if FIELD_PREFIX and not fid.startswith(FIELD_PREFIX):
            continue
        records.append({
            'Field': fid,
            'Description': desc,
            'Type': ftype,
            'Coverage': coverage
        })

    if len(items) < LIMIT:
        break
    PARAMS['offset'] += LIMIT

# 生成 DataFrame 去重並排序
df = pd.DataFrame(records).drop_duplicates().sort_values('Field')

# 輸出 Excel 並嘗試自動調整欄寬
try:
    import xlsxwriter
    with pd.ExcelWriter(OUTPUT_XLSX, engine='xlsxwriter') as writer:
        df.to_excel(writer, index=False, sheet_name='Fields')
        worksheet = writer.sheets['Fields']
        for idx, col in enumerate(df.columns):
            max_len = df[col].astype(str).map(len).max()
            worksheet.set_column(idx, idx, max_len + 2)
    print(f"Excel 已儲存至 {OUTPUT_XLSX}（欄寬已自動調整）")
except ModuleNotFoundError:
    print("未安裝 xlsxwriter，請執行: pip install xlsxwriter，才能生成 Excel 檔並自動調整欄寬。")


Excel 已儲存至 C:\Users\USER\Desktop\雲端同步\WorldQuant\Data\Universe_Dataset_fields.xlsx（欄寬已自動調整）
